# Motion Models
Source: Chapter 5 of Probabilistic Robotics.

## Introduction
We only focus in robots operating in planar environments.
We aim to describe a probabilistic model of kinematics, such that the outcome of a control is described by a posterior probability.
In practice, the exact shape of the model often seems less important that the fact that some provisions for uncertain outcomes are provided in the first place. In fact, many of the most successful models vastly overestimate the amount of uncertainty,


## Kinematic configuration
The kinematic state of robots operating in planar environments can be summarized by three variables $(x, y, \theta)$ describing respectively the position in 2D and the angular orientation.

We decide that $\theta = 0$ means that the robot points in the direction of its x axis.

In [3]:
class State:
    """
    State of a wheeled robot moving in 2D
    """
    def __init__(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta
        self.time = 0

    def __repr__(self):
        return f"State at time {self.time}: [x = {self._x()}, y = {self._y()}, theta = {self._theta()}]"
    
    def _x(self):
        return self.x

    def _y(self):
        return self.y

    def _theta(self):
        return self.theta

    def as_vector(self):
        return [self._x(), self._y(), self._theta()]


class Robot:
    def __init__(self, state: State):
        # State and control of the robot at the 
        # current moment t
        self.state = state
        self.control = [] #TODO define
        self.motion_error_parameters = [None] * 6

    def apply_control(self, ut):
        self.control = ut




## Probabilistic kinematics
The probabilistic kinematic model, or motion model, plays the role of the state estimation model in mobile robotics. This model is the conditional density $$p(x_t | u_t, x_{t-1})$$ where the $x_i$ are the robot poses and the $u_i$ are the motion commands.

We will describe two motion models:
- One assuming that the motion data $u_t$ specifies the velocity commands given to the robot's motors.
- One assuming that one is provided with odometry information.

In practice, odometry models tend to be more accurate than velocity models, for the simple reasons that most commercial robots do not execute velocity commands with the level of accuracy that can be obtained by measuring the rotation of the robot's wheels. However odometry data is only obtained after the fact, which means that it is not suitable for motion planning, such as obstacle avoidance.

# Velocity motion model
The velocity motion model assumes that we can control a robot through two velocities, a rotational and a translational velocity.

We denote the translational velocity at time $t$ by $v_t$ and the rotational velocity at time $t$ by $w_t$. Hence we have $u_t = [v_t, w_t]$.

## Closed form solution
We want to compute the probability $P(x_t | u_t, x_{t-1})$, ie the probability of a state given a control and a previous state, assuming that the control is carried out for a duration $\Delta t$. 
We have 6 motion error parameters parameters $\alpha_1$ to $\alpha_6$ that are specific to the robot and must be estimated.

- The proposed algorithm first calculates the controls of an error-free robot.
- Then the function $\textbf{prob(x,b)}$ models the motion error, ie it computes the probability of $x$ under a zero-centered variable with variance $b$.

In [2]:
from audioop import mul


class VelocityMotionModel():

    def __init__():
        pass

    @classmethod
    def motion_model_velocity(xt, ut, xprev):
        pass
        
    pass